# Training the Spconv
This tutorial aims at twofold: First, we introduce how to construct and train network with spconv and how to glue the SNN neurons and spconv layers together. Second, we introduce how replace the snn neurons using surrogate gradient with Local ZO neurons.
## 1 Introduction for Components
### 1.1 Spconv
Spconv are a pytorch library for sparse convolution, we use for conv layers, batch norm layers and pooling layers.
GitHub homepage: https://github.com/traveller59/spconv
### 1.2 LocalZO.conv_models
Implementations of Spike Neurons (So far, only LIF), that can glue between Spconv layers

## 2 Training the Spconv
### 2.1 Data loading
first, let's load the data using tonic. Details can be found in train_with_torch_nuro.ipynb, so we just skip it here.

In [1]:
# set the gpu device
gpu_idx = '3'
import os
from tonic.datasets import NMNIST
import tonic
from tonic import transforms
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_idx

sensor_size = tonic.datasets.NMNIST.sensor_size
# remove isolated events
# sum a period of events into a frame
frame_transform = transforms.Compose([
    transforms.Denoise(filter_time=10000),
    transforms.ToFrame(sensor_size=sensor_size, time_window=1000),
])  # the output of ToFrame is a tuple of (frame, label), where frame is np.ndarray

trainset = tonic.datasets.NMNIST(save_to='/home/zxh/data', train=True, transform=frame_transform)
testset = tonic.datasets.NMNIST(save_to='/home/zxh/data', train=False, transform=frame_transform)
print('trainset size:', len(trainset), 'testset size:', len(testset), 'type of dataset', type(trainset[0][0]), 'shape', trainset[0][0].shape)

trainset size: 60000 testset size: 10000 type of dataset <class 'numpy.ndarray'> shape (298, 2, 34, 34)


### 2.2 Define the network
The construction of a Spconv network is basically same with constructing any torch.nn networks, except for a few things to notice:
* The computation is time-first, and at the end of LIF layer we reshape the input into [time*batch, *inputs] before feeding into spconv network to improve the performance. So, we need to reshape it back before output.
* The input of a Spconv network is a sparse tensor, constructed by spconv.SparseConvTensor.from_dense()
* The LeakyPlain (implemented in LocalZO.conv_models.neurons) is a spike neuron that can glue between Spconv layers. It needs to specify the batch_size, u_th and beta. Batch size is used to reshape input and output, u_th and beta are parameters of LIF neuron.
* The output should be a tensor with shape (time, batch, *inputs).

**Important:** The spconv library treat empty input (i.e. input with only zeros) as an error
Usually this will not happen, if so, we will get *CUDA kernel launch blocks must be positive, but got N= 0*. and please set appropriate u_th and beta to make at least one spike fire during the time stamps. Details can be found in : https://github.com/traveller59/spconv/blob/master/docs/COMMON_PROBLEMS.md


**Not so Important** if just profile the network with only fc layers, ... just go head like constructing normal torch.nn networks anyway, not forget to reshape it back!

In [2]:
import torch
from torch import nn
from spconv import pytorch as spconv  # pay attention to the spconv.pytorch
from conv_models.neurons import LeakyPlain


class ExampleNet(nn.Module):
    def __init__(self, batch_size, u_th, beta, conv_algorithm = spconv.ConvAlgo.Native):
        super(ExampleNet, self).__init__()
        self.batch_size= batch_size
        self.conv_block1 = nn.Sequential(
            spconv.SparseConv2d(2, 16, 5, bias=True, algo=conv_algorithm),
            spconv.SparseBatchNorm(16, eps=1e-5, momentum=0.1),
            spconv.SparseMaxPool2d(2, stride=2),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size),
        )
        self.conv_block2 = nn.Sequential(
            spconv.SparseConv2d(16, 32, 5, bias=True, algo=conv_algorithm),
            spconv.SparseBatchNorm(32, eps=1e-5, momentum=0.1),
            spconv.SparseMaxPool2d(2, stride=2),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size),
        )
        self.to_dense = spconv.ToDense() # convert the sparse tensor to dense tensor
        self.flatten = nn.Flatten(start_dim=1)
        self.fc = nn.Sequential(
            nn.Linear(32*5*5, 10),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size),
        )

    def forward(self, x):
        assert isinstance(x, spconv.SparseConvTensor)
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.to_dense(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = x.view(-1, self.batch_size, 10)  # reshape back into time, batch, *inputs
        return x

batch_size = 128
net = ExampleNet(batch_size=batch_size, u_th=0.8, beta=0.6).cuda()
print(net)
for name, param in net.named_parameters():
    print(name, param.shape)

ExampleNet(
  (conv_block1): Sequential(
    (0): SparseConv2d(2, 16, kernel_size=[5, 5], stride=[1, 1], padding=[0, 0], dilation=[1, 1], output_padding=[0, 0], algo=ConvAlgo.Native)
    (1): SparseBatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SparseMaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=[0, 0], dilation=[1, 1], algo=ConvAlgo.MaskImplicitGemm)
    (3): LeakyPlain()
  )
  (conv_block2): Sequential(
    (0): SparseConv2d(16, 32, kernel_size=[5, 5], stride=[1, 1], padding=[0, 0], dilation=[1, 1], output_padding=[0, 0], algo=ConvAlgo.Native)
    (1): SparseBatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SparseMaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=[0, 0], dilation=[1, 1], algo=ConvAlgo.MaskImplicitGemm)
    (3): LeakyPlain()
  )
  (to_dense): ToDense()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=800, out_features=10, bias=True)
    (1

### 2.3 Define the loss function and optimizer
we use loss function from snntorch, and optimizer from torch.optim

In [3]:
from snntorch import functional as SF
import torch

loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)
acc_fn = SF.accuracy_rate
optimizer = torch.optim.Adam(net.parameters(), lr=2e-3)

### 2.4 Cache dataset to accelerate training

In [4]:
import torchvision
from torch.utils.data import DataLoader

cache_transform = tonic.transforms.Compose([
    torch.from_numpy,
    torchvision.transforms.RandomRotation([-10,10]),
])

epoch_num = 5
cached_trainset = tonic.DiskCachedDataset(trainset, cache_path='./data/cache', transform=cache_transform)
cached_testset = tonic.DiskCachedDataset(testset, cache_path='./data/cache',)

train_loader = DataLoader(cached_trainset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True, collate_fn=tonic.collation.PadTensors(batch_first=False))
test_loader = DataLoader(cached_testset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=True,
                         collate_fn=tonic.collation.PadTensors(batch_first=False))

### 2.6 Training

In [5]:
from torch.utils.tensorboard import SummaryWriter
from time import time

def train_net_work(net, train_loader, test_loader=None, epoch_num=1, max_iter=200):
    with SummaryWriter(comment='spconv', log_dir='./output/spconv') as writer:
        for epoch in range(epoch_num):
            for i, (inputs, labels) in enumerate(train_loader):
                inputs = inputs.view(-1, *inputs.shape[2:]).transpose(1, 3).cuda()
                inputs = spconv.SparseConvTensor.from_dense(inputs)
                labels = labels.cuda()
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = loss_fn(outputs, labels)
                loss.backward()
                optimizer.step()

                if i % 10 == 0:
                    writer.add_scalar('loss', loss.item(), epoch * len(train_loader) + i)
                    writer.add_scalar('acc', acc_fn(outputs, labels).item(), epoch * len(train_loader) + i)
                    print('epoch', epoch, 'batch', i, 'loss', loss.item(), 'acc', acc_fn(outputs, labels).item())
                if i > max_iter:
                    break

train_net_work(net, train_loader, test_loader, 1)

2023-05-04 22:43:12.713651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 22:43:17.745580: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-05-04 22:43:17.745996: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-05-04 22:43:17.746026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] 

epoch 0 batch 0 loss 26.820280075073242 acc 0.140625
epoch 0 batch 10 loss 13.462841987609863 acc 0.15625
epoch 0 batch 20 loss 9.124115943908691 acc 0.359375
epoch 0 batch 30 loss 7.92097806930542 acc 0.53125


KeyboardInterrupt: 

## 3 Training with LocalZO neurons
### 3.1 Implementation Details
The LocalZO neurons use random sampler to generate random tangents to approximate gradient
Since the sampling technique varies, to instantiate a LocalZo neuron, we need to provide a sampler. For customized sampler, we recommend to inherit the base class `BaseSampler` and implement the `generate_random_tangents` method.

In [6]:
from conv_models.neurons import LeakeyZOPlain
from conv_models.samplers import NormalSampler, BaseSampler


class ExampleNetLocalZO(nn.Module):
    def __init__(self, batch_size, u_th, beta, conv_algorithm = spconv.ConvAlgo.Native, random_sampler: BaseSampler=NormalSampler, sample_num=1):
        super(ExampleNetLocalZO, self).__init__()
        self.batch_size= batch_size
        self.conv_block1 = nn.Sequential(
            spconv.SparseConv2d(2, 16, 5, bias=True, algo=conv_algorithm),
            spconv.SparseBatchNorm(16, eps=1e-5, momentum=0.1),
            spconv.SparseMaxPool2d(2, stride=2),
            LeakeyZOPlain(u_th=u_th, beta=beta, batch_size=batch_size, random_sampler=random_sampler(),sample_num=sample_num)
        )
        self.conv_block2 = nn.Sequential(
            spconv.SparseConv2d(16, 32, 5, bias=True, algo=conv_algorithm),
            spconv.SparseBatchNorm(32, eps=1e-5, momentum=0.1),
            spconv.SparseMaxPool2d(2, stride=2),
            LeakeyZOPlain(u_th=u_th, beta=beta, batch_size=batch_size, random_sampler=random_sampler(),sample_num=sample_num)
        )
        self.to_dense = spconv.ToDense() # convert the sparse tensor to dense tensor
        self.flatten = nn.Flatten(start_dim=1)
        self.fc = nn.Sequential(
            nn.Linear(32*5*5, 10),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size)
        )

    def forward(self, x):
        assert isinstance(x, spconv.SparseConvTensor)
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.to_dense(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = x.view(-1, self.batch_size, 10)  # reshape back into time, batch, *inputs
        return x

net = ExampleNetLocalZO(batch_size=batch_size, u_th=1.0, beta=0.5).cuda()
print(net)
for name, param in net.named_parameters():
    print(name, param.shape)

ExampleNetLocalZO(
  (conv_block1): Sequential(
    (0): SparseConv2d(2, 16, kernel_size=[5, 5], stride=[1, 1], padding=[0, 0], dilation=[1, 1], output_padding=[0, 0], algo=ConvAlgo.Native)
    (1): SparseBatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SparseMaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=[0, 0], dilation=[1, 1], algo=ConvAlgo.MaskImplicitGemm)
    (3): LeakeyZOPlain()
  )
  (conv_block2): Sequential(
    (0): SparseConv2d(16, 32, kernel_size=[5, 5], stride=[1, 1], padding=[0, 0], dilation=[1, 1], output_padding=[0, 0], algo=ConvAlgo.Native)
    (1): SparseBatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SparseMaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=[0, 0], dilation=[1, 1], algo=ConvAlgo.MaskImplicitGemm)
    (3): LeakeyZOPlain()
  )
  (to_dense): ToDense()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=800, out_features=10, bias

In [7]:
from torch.utils.tensorboard import SummaryWriter
import time
from snntorch import functional as SF
import torch

loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)
acc_fn = SF.accuracy_rate
optimizer = torch.optim.Adam(net.parameters(), lr=2e-3)

train_net_work(net, train_loader, test_loader, 1, max_iter=30)

epoch 0 batch 0 loss 29.595436096191406 acc 0.1171875
epoch 0 batch 10 loss 10.910920143127441 acc 0.21875
epoch 0 batch 20 loss 9.111592292785645 acc 0.4609375
epoch 0 batch 30 loss 8.278526306152344 acc 0.453125


## 4 Profile Sparsity
If we need to profile the network, just pass profile=True in the neurons, then we can see the result. The result should be printed in the order of backward pass.

In [8]:
class ProfileNet(nn.Module):
    def __init__(self, batch_size, u_th, beta, conv_algorithm = spconv.ConvAlgo.Native, random_sampler: BaseSampler=NormalSampler, sample_num=1):
        super(ProfileNet, self).__init__()
        self.batch_size= batch_size
        self.conv_block1 = nn.Sequential(
            spconv.SparseConv2d(2, 16, 5, bias=True, algo=conv_algorithm),
            spconv.SparseBatchNorm(16, eps=1e-5, momentum=0.1),
            spconv.SparseMaxPool2d(2, stride=2),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size, profile=True)
        )
        self.conv_block2 = nn.Sequential(
            spconv.SparseConv2d(16, 32, 5, bias=True, algo=conv_algorithm),
            spconv.SparseBatchNorm(32, eps=1e-5, momentum=0.1),
            spconv.SparseMaxPool2d(2, stride=2),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size, profile=True)
        )
        self.to_dense = spconv.ToDense() # convert the sparse tensor to dense tensor
        self.flatten = nn.Flatten(start_dim=1)
        self.fc = nn.Sequential(
            nn.Linear(32*5*5, 10),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size, profile=True)
        )

    def forward(self, x):
        assert isinstance(x, spconv.SparseConvTensor)
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.to_dense(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = x.view(-1, self.batch_size, 10)  # reshape back into time, batch, *inputs
        return x

net = ProfileNet(batch_size=batch_size, u_th=1.0, beta=0.5).cuda()
train_net_work(net, train_loader, test_loader, 1, max_iter=30)

the sparsity of the input grad is tensor(0.2963, device='cuda:0')
the sparsity of the output grad is tensor(0.0383, device='cuda:0')
the sparsity of the input grad is tensor(0.8068, device='cuda:0')
the sparsity of the output grad is tensor(0.4533, device='cuda:0')
epoch 0 batch 0 loss 30.725879669189453 acc 0.0703125
the sparsity of the input grad is tensor(0.3211, device='cuda:0')
the sparsity of the output grad is tensor(0.0390, device='cuda:0')
the sparsity of the input grad is tensor(0.8204, device='cuda:0')
the sparsity of the output grad is tensor(0.4752, device='cuda:0')
the sparsity of the input grad is tensor(0.2917, device='cuda:0')
the sparsity of the output grad is tensor(0.0396, device='cuda:0')
the sparsity of the input grad is tensor(0.7995, device='cuda:0')
the sparsity of the output grad is tensor(0.4449, device='cuda:0')
the sparsity of the input grad is tensor(0.2903, device='cuda:0')
the sparsity of the output grad is tensor(0.0341, device='cuda:0')
the sparsity of

we can see a decrease of sparsity here.